In [17]:
# Dependencies and Setup
#import matplotlib.pyplot as plt
import pandas as pd
#import numpy as np
#import requests
#import time
#from scipy.stats import linregress

import us

# Import API key
#from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
#from citipy import citipy

# pprint - useful for browsing api results
#from pprint import pprint

resource_path = "../../Resources/"
age_bins_path = f"{resource_path}Age Bins.csv"
census_path = f"{resource_path}census_data_output.csv"
land_area_path = f"{resource_path}State_County_MHI_Landarea.csv"

In [5]:
area_df = pd.read_csv(land_area_path)
area_df.head()

,Zip Code,State,County,Median Household Income,Land Area in Square Miles
0,601,PR,Adjuntas Municipio,NaN,NaN
1,602,PR,Aguada Municipio,NaN,30.61
2,603,PR,Aguadilla Municipio,NaN,31.61
3,606,PR,Maricao Municipio,NaN,42.31
4,610,PR,Aasco Municipio,NaN,35.92


In [48]:
area_df['Median Household Income'].unique()

#Eliminate records with no land area or income figures
clean_area_df = area_df[area_df['Land Area in Square Miles'].notnull()]
clean_area_df = clean_area_df[clean_area_df['Median Household Income'].notnull()]

#Missing county name is insignificant for statewide analysis;
#would need to deal with if drilling down into Oklahoma counties
#clean_area_df = clean_area_df[clean_area_df['County'].isnull()]

#Remove any records that reference a negative income; this could be attributed to zip codes with no residences
clean_area_df = clean_area_df[clean_area_df['Median Household Income'] > 0]

#Clean up column names; this will help with merging further below
clean_area_df.rename(columns={'Land Area in Square Miles' : 'Land Area (m2)'}, inplace=True)

#A useable data source without nulls will have same counts for all columns
#(Again, the missing county is insignificant to statewide aggregation)
clean_area_df.count()

Zip Code                   31859
State                      31859
County                     31858
Median Household Income    31859
Land Area (m2)             31859
dtype: int64

In [26]:
#If the result of this expression matches the total row count above,
#then the column values are all unique and zip code can be used to merge
print(len(clean_area_df['Zip Code'].unique()))

31859


In [20]:
census_df = pd.read_csv(census_path)
census_df.head()

,Population,Median Household Income,Per Capita Income,Zipcode,Population density
0,17242.0,13092.0,6999.0,601,0.000053
1,38442.0,16358.0,9277.0,602,0.000118
2,48814.0,16603.0,11307.0,603,0.000150
3,6437.0,12832.0,5943.0,606,0.000020
4,27073.0,19309.0,10220.0,610,0.000083


In [55]:
#Eliminate rows that are missing data for any measures (income, population, etc.)
clean_census_df = census_df[census_df['Median Household Income'].notnull()]
clean_census_df = clean_census_df[clean_census_df['Per Capita Income'].notnull()]

#Remove any records that reference a negative income; this could be attributed to zip codes with no residences
clean_census_df = clean_census_df[clean_census_df['Median Household Income'] > 0]
clean_census_df = clean_census_df[clean_census_df['Per Capita Income'] > 0]

clean_census_df['Total Income'] = clean_census_df['Per Capita Income'] * clean_census_df['Population']

#Clean up column names; this will help with merging further below
clean_census_df.rename(columns={'Zipcode' : 'Zip Code',
                                'Population density' : 'Population Density'}, inplace=True)

#With null measures eliminated, they should all return the same count
clean_census_df.count()

Population                 30888
Median Household Income    30888
Per Capita Income          30888
Zip Code                   30888
Population Density         30888
Total Income               30888
dtype: int64

In [56]:
#If the result of this expression matches the total row count above,
#then the column values are all unique and zipcode can be used to merge
print(len(clean_census_df['Zip Code'].unique()))

30888


In [57]:
merge_df = pd.merge(clean_area_df, clean_census_df, on='Zip Code', how='inner', suffixes=('_a', '_b'), validate='one_to_one')
#merge_df.count()

clean_merge_df = merge_df[merge_df['State'].notnull()]
clean_merge_df = clean_merge_df[clean_merge_df['Population'].notnull()]

clean_merge_df.count()

Zip Code                     30636
State                        30636
County                       30635
Median Household Income_a    30636
Land Area (m2)               30636
Population                   30636
Median Household Income_b    30636
Per Capita Income            30636
Population Density           30636
Total Income                 30636
dtype: int64

In [62]:
state_grp = clean_merge_df.groupby(['State'])
state_mhia = state_grp['Median Household Income_a'].median()
state_mhib = state_grp['Median Household Income_b'].median()
state_area = state_grp['Land Area (m2)'].sum()
state_pop = state_grp['Population'].sum()
state_pci = state_grp['Total Income'].sum() / state_pop
state_pd = state_pop / state_grp['Land Area (m2)'].sum()

state_df = pd.DataFrame(state_area)
state_df['Population'] = state_pop
state_df['Population Density'] = state_pd
state_df['Per Capita Income'] = state_pci
#state_df['Median Household Income A'] = state_mhia
state_df['Median Household Income'] = state_mhib

state_df.rename(columns={'Land Area (m2)' : 'Land Area (mi2)'}, inplace=True)

state_df

,Land Area (mi2),Population,Population Density,Per Capita Income,Median Household Income
State,,,,,
AK,211300.81,734417.0,3.475694,35617.249121,51875.0
AL,49481.34,4846150.0,97.938940,26883.102656,41750.0
AR,49590.60,2977621.0,60.044061,25666.556534,40574.0
AZ,89709.56,6940873.0,77.370494,29270.505603,49063.0
CA,96922.44,39076841.0,403.176406,35049.472573,65231.0
CO,97067.89,5514222.0,56.807890,36469.426840,59610.5
CT,4829.57,3569498.0,739.092300,43172.210523,86324.0
DC,59.58,677046.0,11363.645519,53842.673755,88553.0
DE,1945.37,943604.0,485.051173,34155.262572,61657.0


In [63]:
state_df.to_csv("Resources/IncomePopByState.csv")

In [64]:
age_df = pd.read_csv(age_bins_path)
age_df.head()

,Age Bracket,Number of People,Percent of people
0,Under 5 years,"19,929",6.2
1,5 to 9 years,"20,243",6.3
2,10 to 14 years,"20,812",6.4
3,15 to 19 years,"20,908",6.5
4,20 to 24 years,"21,434",6.6
